Global Runoff Data Center (GRDC) https://portal.grdc.bafg.de/

Purpose: produce maps of GRDC gauge sites and upstream watersheds

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import codebase

- below Hoover Dam: 4152103
- Lee's Ferry: 4152450
- US-Mexico border: 4152050
- upstream of Lake Powell (San Juan River trib): 4152600

In [2]:
IDs = [4152103, 4152450, 4152050, 4152600]

In [3]:
datadir = '/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/'

In [4]:
res_shp = codebase.load_data.load_GRanD()

In [5]:
lake_mead = codebase.area_subsets.check_for_multiple_dams('hoover', res_shp)
lake_powell = codebase.area_subsets.check_for_multiple_dams('glen canyon', res_shp)

In [6]:
subregions_meta = gpd.read_file(datadir + 'colorado_subregions.geojson')
stations_meta = gpd.read_file(datadir + 'colorado_stationbasins.geojson')
stations_meta.set_index('grdc_no',inplace=True)
stations_meta.head(2)

,river,station,area,altitude,lat_org,long_org,lat_pp,long_pp,dist_km,area_calc,quality,type,comment,source,geometry
grdc_no,,,,,,,,,,,,,,,
4152050.0,COLORADO RIVER,"BELOW YUMA MAIN CANAL WW AT YUMA,AZ",618715.0,31.09,32.7317,-114.6319,32.7313,-114.6312,0.1,615846.2,High,Automatic,Area difference <= 5% and distance <= 5 km,hydrosheds,"MULTIPOLYGON (((-110.90830 41.35830, -110.9042..."
4152100.0,COLORADO RIVER,"YUMA, ARIZ.",609679.0,31.35,32.7292,-114.6208,32.7313,-114.6188,0.3,615843.6,High,Automatic,Area difference <= 5% and distance <= 5 km,hydrosheds,"MULTIPOLYGON (((-110.90830 41.35830, -110.9042..."


In [7]:
stations_polys = stations_meta.loc[IDs]

In [8]:
stations_polys['lat_lon'] = gpd.points_from_xy(y =stations_polys['lat_org'], x =  stations_polys['long_org'])
stations_points = stations_polys.set_geometry('lat_lon', crs = "EPSG:4326")

In [9]:
stations_points.explore()

In [10]:
m = stations_polys.explore(style_kwds=dict(color='green',opacity=0.2, fillOpacity=.2))
lake_mead.explore(m=m,style_kwds=dict(color='blue'))
lake_powell.explore(m=m,style_kwds=dict(color='blue'))
# stations_points.explore(m=m,style_kwds=dict(fillcolor='fuchisa',color='fuchsia',size = 250))
m

In [11]:
test_ID = 4152050

In [13]:
test_gpd, test_df = codebase.load_data.load_GRDC_station_data_by_ID(test_ID,filepath=datadir, basin_str='colorado')

In [15]:
test_gpd.head()

,grdc_no,river,station,area,altitude,lat_org,long_org,lat_pp,long_pp,dist_km,area_calc,quality,type,comment,source,geometry
0,4152050.0,COLORADO RIVER,"BELOW YUMA MAIN CANAL WW AT YUMA,AZ",618715.0,31.09,32.7317,-114.6319,32.7313,-114.6312,0.1,615846.2,High,Automatic,Area difference <= 5% and distance <= 5 km,hydrosheds,"MULTIPOLYGON (((-110.90830 41.35830, -110.9042..."
